In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
# List all test image filenames and sort them
test_folder = r'C:\Users\agnao\OneDrive\Desktop\workshop AI\Dog_vs_Cat_CNN_Project\test'
filenames = os.listdir(test_folder)
filenames.sort()  # ensures 1.jpg, 2.jpg, ... 12500.jpg order

# Create DataFrame for the generator
test_df = pd.DataFrame({'filename': filenames})

In [ ]:
#data preprocessing
train_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)  # 20% for validation
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
                  r'C:\Users\agnao\OneDrive\Desktop\workshop AI\Dog_vs_Cat_CNN_Project\train',
                  target_size =(100, 100),  # target_size = input image size
                  batch_size = 32,
                  class_mode ='binary',
                  subset='training')

validation_generator = train_datagen.flow_from_directory(
                  r'C:\Users\agnao\OneDrive\Desktop\workshop AI\Dog_vs_Cat_CNN_Project\train',
                  target_size =(100, 100), 
                  batch_size = 32,
                  class_mode ='binary',
                  subset='validation')


test_generator = test_datagen.flow_from_dataframe(
                    dataframe=test_df,
                    directory=r'C:\Users\agnao\OneDrive\Desktop\workshop AI\Dog_vs_Cat_CNN_Project\test',
                    x_col='filename',
                    target_size =(100, 100),
                    batch_size = 32,
                    class_mode =None,
                    shuffle=False
                    )

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 validated image filenames.


In [4]:
#building model
import random
options={
    "filters_per_layer_sets" : [[64, 128, 256],[64,128,256,512]],
    'dense_units': [256, 512],
    'learning_rate': [1e-3, 7.5e-4],
    'L2_weight': [0.01, 0.005]
}

def select_hyper_par(options):
    set_of_filters = random.choice(options["filters_per_layer_sets"])
    dense_units=random.choice(options["dense_units"])
    learning_rate=random.choice(options["learning_rate"])
    L2_weight=random.choice(options["L2_weight"])

    return{
        "num_of_conv_layers":len(set_of_filters),
        "filters_per_layer":set_of_filters,
        'dense_units': dense_units,
        'learning_rate': learning_rate,
        'L2_weight':L2_weight
    }

def build_model(options):
    model = Sequential()

    filters=options["filters_per_layer"]
    dense_units=options["dense_units"]
    learning_rate=options["learning_rate"]
    L2_weight=options["L2_weight"]

    for i,f in enumerate(filters):
        if i == 0:
            model.add(Conv2D(f,(3,3), input_shape=(100, 100, 3)))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
        else:
            model.add(Conv2D(f,(3,3)))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(dense_units))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation='sigmoid',kernel_regularizer =tf.keras.regularizers.l2(L2_weight)))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:

# we test multiple model using validation_generator
search_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.8  # Use 20% of data for search
)

# Small subset
search_generator = search_datagen.flow_from_directory(
    directory=r'C:\Users\agnao\OneDrive\Desktop\workshop AI\Dog_vs_Cat_CNN_Project\train',
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed=42
)
def Random_search(n_iter):
    best_score = 0
    best_params = None

    for i in range(n_iter):
        params=select_hyper_par(options)
        model = build_model(params)
        history = model.fit(
                    search_generator,
                    steps_per_epoch = 40, #num_train_samples // batch_size
                    epochs = 10,
                    validation_data = validation_generator,
                    validation_steps=20,
                    verbose=0,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])
        # Mandatory reset
        search_generator.reset()
        validation_generator.reset()

        val_acc = max(history.history['val_accuracy'])
        val_loss = min(history.history['val_loss'])

        # Better scoring metric (higher is better)
        score = val_acc / (1 + val_loss)

        if i == 0 or score > best_score:
            best_score = score
            best_params = params
    return best_params,best_score
params,best_score = Random_search(10)
print(params)
print(best_score)

Found 5000 images belonging to 2 classes.
{'num_of_conv_layers': 3, 'filters_per_layer': [64, 128, 256], 'dense_units': 256, 'learning_rate': 0.001, 'L2_weight': 0.01}
0.3784970218333528


In [ ]:
#best params after multiple random search
best_params = {'num_of_conv_layers': 4, 'filters_per_layer': [64, 128, 256, 512], 'dense_units': 512, 'learning_rate': 0.001, 'L2_weight': 0.01}

In [ ]:

# Your compiled model being trained with fit_generator
model = build_model(params)
history = model.fit(
             train_generator,
             steps_per_epoch = 625, # num_train_samples // batch_size
             epochs = 20,
             validation_data = validation_generator,
             validation_steps = 156, # num_validation_samples // batch_size
             callbacks=[early_stop])

In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator))


391/391 ━━━━━━━━━━━━━━━━━━━━ 52s 132ms/step


In [ ]:
predicted_classes = (predictions > 0.5).astype(int)


In [ ]:

# Add predictions to DataFrame
test_df['id'] = test_df['filename'].str.replace('.jpg', '').astype(int)
test_df['label'] = predicted_classes.flatten()

# Create final submission DataFrame and save CSV
submission_df = test_df[['id', 'label']].sort_values('id')  # ensure it's in correct id order
submission_df.to_csv('final_submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

plot_history(history)